In [6]:
import pandas as pd
import numpy as np
import sqlite3


# 連接到資料庫A ====================================================================
conn = sqlite3.connect('goodinfoRevenue.db')
cursor = conn.cursor()
sql = '''select * from revenue t'''
df = pd.read_sql(sql,conn)

# 連接到資料庫B
sqlStock = '''select * from stock t'''
dfStock = pd.read_sql(sqlStock,conn)
dfStock = pd.DataFrame(dfStock)
ids = dfStock['code']
# ==================================================================================


# 複製DataFrame並更改欄位名稱
df2 = df
df2.columns = ['代碼','月別','開盤','收盤','最高','最低','漲跌(元)','漲跌(%)','營收(億)','月增(%)','年增(%)','累計營收(億)','累計年增(%)']

# 創建空的DataFrame
dataExport = pd.DataFrame(columns = ['代碼','月別','開盤','收盤','最高','最低','漲跌(元)','漲跌(%)','營收(億)','月增(%)','年增(%)','累計營收(億)','累計年增(%)','當月營收排名','累計排名'])
# count = 0

for id in ids:
    # print(f'現在在{count}行,股票代碼為{id}')
    df2mask = df2['代碼'] == id
    df3 = df2[df2mask]
    df3 = df3.sort_values(by='月別', ascending=False)

    # 當月營收排名 ==============================================
    df3.insert(13,'當月營收排名',df3['營收(億)'].rank(ascending=False))

    # 當月累計排名 ==============================================
    當月 = df3.iloc[0]['月別'].split('/',1) # 去除年份取月份
    當月 = 當月[1]
    累計 = df3['月別'].str.contains(當月,na=False) # 取出所有同期月份，並為同期月份進行排名
    df3.insert(14,'累計排名',df3[累計]['累計營收(億)'].rank(ascending=False))
    # 去除NaN ===================================================
    df3 = df3.fillna(0)
    # ============================================================
    dataExport = pd.concat([dataExport,df3],axis=0,ignore_index = True)
    # count+=1
print('已整理完營收統計資料,請導出')

已整理完營收統計資料,請導出


# 導出CSV

In [7]:
dataExport.to_csv(f"goodinfo營收統計.csv", encoding="big5hkscs",index=False)
print('導出成功')

導出成功


# 查找個股

In [2]:
maskStock = dataExport['代碼'] == '6523'
dataExportSearch = dataExport[maskStock]
dataExportSearch

,代碼,月別,開盤,收盤,最高,最低,漲跌(元),漲跌(%),營收(億),月增(%),年增(%),累計營收(億),累計年增(%),當月營收排名,累計排名
64669,6523,2022/07,86.3,82.1,86.9,79.0,-4.3,-4.98,0.934,47.7,0.73,5.170,-8.64,12.0,4.0
64670,6523,2022/06,87.5,86.4,91.2,79.7,0.4,0.47,0.632,1.9,-14.4,4.230,-10.5,46.5,0.0
64671,6523,2022/05,80.2,86.0,87.3,75.6,5.8,7.23,0.621,53.2,-18.5,3.600,-9.75,51.0,0.0
64672,6523,2022/04,92.0,80.2,93.4,79.0,-12.0,-13.02,0.405,-45.4,-52.5,2.980,-7.7,55.0,0.0
64673,6523,2022/03,99.5,92.2,103.0,92.0,-7.0,-7.06,0.741,12.5,10.6,2.570,8.36,34.0,0.0
64674,6523,2022/02,89.7,99.2,101.0,89.7,10.2,11.46,0.659,-43.8,-14.7,1.830,7.46,42.0,0.0
64675,6523,2022/01,101.5,89.0,105.5,86.7,-11.5,-11.44,1.170,62.8,25.8,1.170,25.8,5.0,0.0
64676,6523,2021/12,146.5,100.5,152.5,98.0,-46.5,-31.63,0.721,-35.9,15.9,11.060,44.6,37.0,0.0
64677,6523,2021/11,95.0,147.0,151.0,80.0,53.0,56.38,1.120,-4.13,36.4,10.340,47.2,7.0,0.0
64678,6523,2021/10,84.7,94.0,94.5,77.3,9.8,11.64,1.170,-0.06,80.5,9.220,48.6,5.0,0.0
